In [54]:
from bs4 import BeautifulSoup
import requests
import os.path
from htmldate import find_date
import pandas as pd
def scrape(url):
    try:
        response = requests.get(url)
    except Exception:
        return "DELETE"
    html_content = response.text
    soup = BeautifulSoup(html_content, 'lxml')  # You can also use 'html.parser'
    paragraphs = soup.find_all("p")
    testArticle= soup.find_all("a")
    article = True
    for t in testArticle:
        try:
            if t["href"] =="#comments":
                return "DELETE"
        except Exception:
            continue
    
    textOut = ""
    try:
        timeStamp = find_date(url)
        year = timeStamp.split("-")[0]
        month = timeStamp.split("-")[1]
        day = timeStamp.split("-")[2]
    except Exception:
        year = "check"
        month = "check"
        day = "check"
    if year == "check":
        findDate = url.split("/")
        dateList = []
        for index,f in enumerate(findDate):
            if f in [2016,2017,2018,2019,2020,2021]:
                for i in range(index,index+3):
                        dateList.append(findDate[i])
                        print(dateList[-1])
                year = dateList[0]
                month = dateList[1]
                day = dateList[2]
                break
        
    removeList = []
    remove = soup.find_all("div",class_="mb-2 text-blurple [&>p>span:first-child]:text-gray-13 [&_.duet--article-byline-and]:text-gray-13")
    moreRemove = soup.find_all("div",{"class":"mb-2"})
    for m in moreRemove:
        removeList.append(m.get_text().strip().replace("\n",""))
    removeList = []
    for r in remove:
        removeList.append(r.get_text().strip().replace("\n",""))
    text = soup.find_all("p")
    removeList.extend(["/ Sign up for Verge Deals to get deals on products we've tested sent to your inbox weekly.","The Verge is a vox media network","© 2024 Vox Media, LLC. All Rights Reserved",
                        "If you buy something from a Verge link, Vox Media may earn a commission. See our ethics statement.","The interview has been condensed and edited."])
    for block,p in enumerate(paragraphs):
        text = p.get_text().replace("\n","").strip()
        if text != "" and text[-1] in '!.?:)”"' and text not in removeList and  not (block==0 and "By  " in text) and ("Sign up here." not in text and "Platformer" not in text) and not (text[0:10]=="Correction" and block!=len(paragraphs)-1 and paragraphs[block+1].get_text().strip().replace("\n","") in ["/ Sign up for Verge Deals to get deals on products we've tested sent to your inbox weekly.","The Verge is a vox media network","© 2024 Vox Media, LLC. All Rights Reserved"]): #this varies between websites since some write in a more academic manner
            textOut+= " " + text
    
    length = 0
    if textOut == "":
        return "DELETE"
    else:
        length = len(textOut.split())
        return [url,"Wired",year,month,day,length,textOut]
df = []
